In [1]:
from pulp import *
import pandas as pd

In [2]:
#Extracting breakfast menu
df = pd.read_csv("/home/vivek/Documents/Data_Science/Data_Sets/menu_updated1.csv")
#List of menu items for breakfast
menu_items = df['Item'].tolist()
print(menu_items)

['Egg McMuffin', 'Egg White Delight', 'Sausage McMuffin', 'Sausage McMuffin with Egg', 'Sausage McMuffin with Egg Whites', 'Steak & Egg McMuffin', 'Bacon, Egg & Cheese Biscuit (Regular Biscuit)', 'Bacon, Egg & Cheese Biscuit (Large Biscuit)', 'Bacon, Egg & Cheese Biscuit with Egg Whites (Regular Biscuit)', 'Bacon, Egg & Cheese Biscuit with Egg Whites (Large Biscuit)', 'Sausage Biscuit (Regular Biscuit)', 'Sausage Biscuit (Large Biscuit)', 'Sausage Biscuit with Egg (Regular Biscuit)', 'Sausage Biscuit with Egg (Large Biscuit)', 'Sausage Biscuit with Egg Whites (Regular Biscuit)', 'Sausage Biscuit with Egg Whites (Large Biscuit)', 'Southern Style Chicken Biscuit (Regular Biscuit)', 'Southern Style Chicken Biscuit (Large Biscuit)', 'Steak & Egg Biscuit (Regular Biscuit)', 'Bacon, Egg & Cheese McGriddles', 'Bacon, Egg & Cheese McGriddles with Egg Whites', 'Sausage McGriddles', 'Sausage, Egg & Cheese McGriddles', 'Sausage, Egg & Cheese McGriddles with Egg Whites', 'Bacon, Egg & Cheese Bagel

In [3]:
#Extracting the protein data for objective function
protein = dict(zip(df['Item'].tolist(), df['Protein'].tolist()))

In [4]:
#Nutritrient labels
nutrition_lables  = list(df)[3::]
print(nutrition_lables)

['Serving Size', 'Calories', 'Calories from Fat', 'Total Fat', 'Total Fat (% Daily Value)', 'Saturated Fat', 'Saturated Fat (% Daily Value)', 'Trans Fat', 'Unsaturated Fat', 'Cholesterol', 'Cholesterol (% Daily Value)', 'Sodium', 'Sodium (% Daily Value)', 'Carbohydrates', 'Carbohydrates (% Daily Value)', 'Dietary Fiber', 'Dietary Fiber (% Daily Value)', 'Sugars', 'Vitamin A (% Daily Value)', 'Vitamin C (% Daily Value)', 'Calcium (% Daily Value)', 'Iron (% Daily Value)', 'Vitamin E (% Daily Value)', 'Vitamin D (% Daily Value)', 'Vitamin K (% Daily Value)', 'Vitamin B12 (% Daily Value)', 'Zinc']


In [5]:
#Constraints based on individual nutrient values
Capacity = {'Serving Size': 20, 'Calories': 1400, 'Calories from Fat': 600, 'Total Fat': 80,
                         'Total Fat (% Daily Value)': 80, 'Saturated Fat': 300.0,
                         'Saturated Fat (% Daily Value)': 180, 'Trans Fat': 10, 'Unsaturated Fat': 30,'Cholesterol': 600,
                        'Cholesterol (% Daily Value)': 200, 'Sodium': 2500, 'Sodium (% Daily Value)': 100,
                         'Carbohydrates': 115, 'Carbohydrates (% Daily Value)': 50, 'Dietary Fiber': 190,
                         'Dietary Fiber (% Daily Value)': 150, 'Sugars': 280,
                         'Vitamin A (% Daily Value)': 18, 'Vitamin C (% Daily Value)': 16, 'Calcium (% Daily Value)': 31,
                         'Iron (% Daily Value)': 40, 'Vitamin E (% Daily Value)': 22, 'Vitamin D (% Daily Value)': 24,
                         'Vitamin K (% Daily Value)': 22,'Vitamin B12 (% Daily Value)': 20,'Zinc': 41
}

In [6]:
ResourceLimitation = []
for i in range(len(nutrition_lables)):
    ResourceLimitation.append(df.iloc[:, 3+i].tolist())
print("Length Nutrition Labels are:", len(nutrition_lables))

('Length Nutrition Labels are:', 27)


In [7]:
ResourceLimitation = makeDict([nutrition_lables,menu_items],ResourceLimitation)
#print("Resourse Limitation:", ResourceLimitation)

In [8]:
#Intializing maximization problem
prob = LpProblem("McDonald's Dinner Meal Protien count",LpMaximize)


In [9]:
#Making the menu items variable as binary
vars = LpVariable.dicts("Is",menu_items,0,None, LpBinary)

In [10]:
#Objective function
prob += lpSum([protein[c] * vars[c] for c in menu_items])


In [11]:
#27 contraints for each nutrient label
for r in nutrition_lables:
    prob += lpSum([ResourceLimitation[r][c]*vars[c] for c in menu_items])<=Capacity[r]

print(prob)

McDonald's Dinner Meal Protien count:
MAXIMIZE
30*Is_Bacon,_Egg_&_Cheese_Bagel + 30*Is_Bacon,_Egg_&_Cheese_Bagel_with_Egg_Whites + 19*Is_Bacon,_Egg_&_Cheese_Biscuit_(Large_Biscuit) + 19*Is_Bacon,_Egg_&_Cheese_Biscuit_(Regular_Biscuit) + 20*Is_Bacon,_Egg_&_Cheese_Biscuit_with_Egg_Whites_(Large_Biscuit) + 20*Is_Bacon,_Egg_&_Cheese_Biscuit_with_Egg_Whites_(Regular_Biscuit) + 19*Is_Bacon,_Egg_&_Cheese_McGriddles + 20*Is_Bacon,_Egg_&_Cheese_McGriddles_with_Egg_Whites + 28*Is_Big_Breakfast_(Large_Biscuit) + 28*Is_Big_Breakfast_(Regular_Biscuit) + 26*Is_Big_Breakfast_with_Egg_Whites_(Large_Biscuit) + 26*Is_Big_Breakfast_with_Egg_Whites_(Regular_Biscuit) + 36*Is_Big_Breakfast_with_Hotcakes_(Large_Biscuit) + 36*Is_Big_Breakfast_with_Hotcakes_(Regular_Biscuit) + 35*Is_Big_Breakfast_with_Hotcakes_and_Egg_Whites_(Large_Biscuit) + 35*Is_Big_Breakfast_with_Hotcakes_and_Egg_Whites_(Regular_Biscuit) + 6*Is_Cinnamon_Melts + 17*Is_Egg_McMuffin + 18*Is_Egg_White_Delight + 5*Is_Fruit_&_Maple_Oatmeal + 5*I

In [12]:
prob.writeLP("McDonaldsBreakMenu_Recommendation.lp")

In [14]:
prob.solve()
print("\n\nStatus:", LpStatus[prob.status])

('\n\nStatus:', 'Optimal')


In [15]:
for v in prob.variables():
    print(v.name, "=", v.varValue)

print("Maximum Protein intake would be = ", value(prob.objective))

('Is_Bacon,_Egg_&_Cheese_Bagel', '=', 0.0)
('Is_Bacon,_Egg_&_Cheese_Bagel_with_Egg_Whites', '=', 0.0)
('Is_Bacon,_Egg_&_Cheese_Biscuit_(Large_Biscuit)', '=', 0.0)
('Is_Bacon,_Egg_&_Cheese_Biscuit_(Regular_Biscuit)', '=', 0.0)
('Is_Bacon,_Egg_&_Cheese_Biscuit_with_Egg_Whites_(Large_Biscuit)', '=', 0.0)
('Is_Bacon,_Egg_&_Cheese_Biscuit_with_Egg_Whites_(Regular_Biscuit)', '=', 0.0)
('Is_Bacon,_Egg_&_Cheese_McGriddles', '=', 0.0)
('Is_Bacon,_Egg_&_Cheese_McGriddles_with_Egg_Whites', '=', 0.0)
('Is_Big_Breakfast_(Large_Biscuit)', '=', 0.0)
('Is_Big_Breakfast_(Regular_Biscuit)', '=', 0.0)
('Is_Big_Breakfast_with_Egg_Whites_(Large_Biscuit)', '=', 0.0)
('Is_Big_Breakfast_with_Egg_Whites_(Regular_Biscuit)', '=', 0.0)
('Is_Big_Breakfast_with_Hotcakes_(Large_Biscuit)', '=', 0.0)
('Is_Big_Breakfast_with_Hotcakes_(Regular_Biscuit)', '=', 0.0)
('Is_Big_Breakfast_with_Hotcakes_and_Egg_Whites_(Large_Biscuit)', '=', 0.0)
('Is_Big_Breakfast_with_Hotcakes_and_Egg_Whites_(Regular_Biscuit)', '=', 0.0)
('Is